In [4]:
"""
ANÁLISIS DE ESTABILIDAD Y CORRELACIÓN DE TENDENCIAS
====================================================
Validación de sensores Mi Flora para sistema de monitoreo distribuido

Métricas:
1. CV (Coeficiente de Variación): Estabilidad temporal por sensor
2. Spearman ρ: Correlación de tendencias entre sensores
"""

import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import warnings
warnings.filterwarnings('ignore')

from google.colab import files

print("\n" + "="*70)
print("  VALIDACIÓN DE SENSORES MI FLORA")
print("  Análisis de Estabilidad y Correlación de Tendencias")
print("="*70 + "\n")

print("Sube los 4 archivos CSV de validación cruzada:\n")
uploaded = files.upload()

if not uploaded:
    print("No se cargaron archivos.")
    raise SystemExit

# =============================================================================
# CONFIGURACIÓN
# =============================================================================

NOMBRES_PARAMETROS = {
    'temperature': 'Temperatura (°C)',
    'moisture': 'Humedad (%)',
    'light': 'Luz (lux)',
    'conductivity': 'Conductividad (µS/cm)'
}

SESIONES_INFO = {
    'miflora_log_validacion_cruzada_1': 'Sesión 1: Mañana - Sin malla',
    'miflora_log_validacion_cruzada_2': 'Sesión 2: Tarde - Sin malla',
    'miflora_log_validacion_cruzada_3': 'Sesión 3: Mañana - Malla 50%',
    'miflora_log_validacion_cruzada_4': 'Sesión 4: Tarde - Malla 50%'
}

# =============================================================================
# FUNCIONES DE ANÁLISIS
# =============================================================================

def calcular_cv(serie):
    """
    Calcula el Coeficiente de Variación (CV) de una serie.
    CV = (σ / μ) × 100%
    """
    serie = serie.dropna()
    if len(serie) < 2:
        return np.nan
    media = serie.mean()
    if media == 0:
        return np.nan
    return (serie.std() / media) * 100


def calcular_spearman_promedio(df_pivot):
    """
    Calcula la correlación de Spearman promedio entre todos los pares de sensores.
    ρ = 1 - (6 Σd²) / (n(n²-1))
    """
    sensores = df_pivot.columns.tolist()
    correlaciones = []

    for i in range(len(sensores)):
        for j in range(i+1, len(sensores)):
            s1, s2 = sensores[i], sensores[j]
            datos = df_pivot[[s1, s2]].dropna()

            if len(datos) < 5:
                continue

            if datos[s1].std() == 0 or datos[s2].std() == 0:
                continue

            rho, p_value = spearmanr(datos[s1], datos[s2])

            if not np.isnan(rho):
                correlaciones.append({
                    'par': f"{s1} ↔ {s2}",
                    'rho': rho,
                    'p_value': p_value
                })

    return correlaciones


def analizar_sesion(df, nombre_sesion):
    """Analiza una sesión: CV por sensor y Spearman entre pares."""

    print(f"\n{'='*70}")
    print(f" {nombre_sesion}")
    print(f"{'='*70}")

    df = df.copy().sort_values('timestamp').reset_index(drop=True)
    sensores = sorted(df['sensor_id'].unique().tolist())
    n_sensores = len(sensores)

    print(f"Registros: {len(df)} | Sensores: {n_sensores}")
    print(f"Período: {df['timestamp'].min().strftime('%Y-%m-%d %H:%M')} → {df['timestamp'].max().strftime('%H:%M')}\n")

    df['ronda'] = df.index // n_sensores

    parametros = ['temperature', 'moisture', 'light', 'conductivity']
    resultados = {}

    for param in parametros:
        if df[param].isna().all():
            continue

        nombre = NOMBRES_PARAMETROS[param]
        print(f"{'─'*50}")
        print(f"📊 {nombre}")
        print(f"{'─'*50}")

        # --- CV por sensor ---
        cvs = []
        print("\n▸ Estabilidad (CV por sensor):")
        for sensor in sensores:
            datos = df[df['sensor_id'] == sensor][param]
            cv = calcular_cv(datos)
            media = datos.mean()
            std = datos.std()

            if not np.isnan(cv):
                cvs.append(cv)
                print(f"   {sensor}: CV = {cv:.2f}%  (μ = {media:.2f} ± {std:.2f})")

        cv_promedio = np.mean(cvs) if cvs else np.nan
        print(f"\n   → CV promedio: {cv_promedio:.2f}%")

        # --- Spearman entre pares ---
        df_pivot = df.pivot_table(
            index='ronda',
            columns='sensor_id',
            values=param,
            aggfunc='mean'
        ).dropna()

        correlaciones = calcular_spearman_promedio(df_pivot)

        rho_promedio = np.nan
        if correlaciones:
            print("\n▸ Correlación de tendencias (Spearman ρ):")
            for c in correlaciones:
                sig = "***" if c['p_value'] < 0.001 else ("**" if c['p_value'] < 0.01 else ("*" if c['p_value'] < 0.05 else ""))
                print(f"   {c['par']}: ρ = {c['rho']:.3f}{sig}")

            rho_promedio = np.mean([c['rho'] for c in correlaciones])
            print(f"\n   → ρ promedio: {rho_promedio:.3f}")
        else:
            print("\n▸ Spearman: No calculable (datos constantes)")

        resultados[param] = {
            'cv_promedio': cv_promedio,
            'rho_promedio': rho_promedio,
            'n_muestras': len(df_pivot)
        }
        print()

    return resultados


# =============================================================================
# PROCESAR ARCHIVOS
# =============================================================================

todos_resultados = []
parametros = ['temperature', 'moisture', 'light', 'conductivity']

for fn in sorted(uploaded.keys()):
    try:
        df = pd.read_csv(fn, header=None)
        df.columns = ['timestamp', 'sensor_id', 'temperature', 'moisture',
                     'light', 'conductivity', 'battery']

        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
        for col in ['temperature', 'moisture', 'light', 'conductivity']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        df = df.dropna(subset=['timestamp', 'sensor_id'])
        df['sensor_id'] = df['sensor_id'].astype(str).str.strip()

        base_name = fn.replace('.csv', '').split('(')[0].strip()
        nombre_sesion = SESIONES_INFO.get(base_name, base_name)

        resultado = analizar_sesion(df, nombre_sesion)
        resultado['sesion'] = nombre_sesion
        todos_resultados.append(resultado)

    except Exception as e:
        print(f"❌ Error en {fn}: {e}")


# =============================================================================
# RESULTADOS FINALES
# =============================================================================

print("\n" + "="*70)
print("RESULTADOS FINALES")
print("="*70)

resumen = []
for param in parametros:
    cvs = [r[param]['cv_promedio'] for r in todos_resultados if param in r and not np.isnan(r[param]['cv_promedio'])]
    rhos = [r[param]['rho_promedio'] for r in todos_resultados if param in r and not np.isnan(r[param]['rho_promedio'])]

    resumen.append({
        'Parámetro': NOMBRES_PARAMETROS[param],
        'CV (%)': np.mean(cvs) if cvs else np.nan,
        'Spearman ρ': np.mean(rhos) if rhos else np.nan
    })

df_resumen = pd.DataFrame(resumen)

print("\n" + "─"*70)
print("TABLA DE RESULTADOS")
print("─"*70)
print("""
┌─────────────────────────┬──────────┬─────────────┐
│ Parámetro               │ CV (%)   │ Spearman ρ  │
├─────────────────────────┼──────────┼─────────────┤""")

for _, row in df_resumen.iterrows():
    param = row['Parámetro']
    cv = row['CV (%)']
    rho = row['Spearman ρ']

    cv_str = f"{cv:.2f}" if not np.isnan(cv) else "--"
    rho_str = f"{rho:.2f}" if not np.isnan(rho) else "--"

    print(f"│ {param:<23} │ {cv_str:>8} │ {rho_str:>11} │")

print("└─────────────────────────┴──────────┴─────────────┘")

print("\n" + "="*70)
print("✅ ANÁLISIS COMPLETADO")
print("="*70 + "\n")


  VALIDACIÓN DE SENSORES MI FLORA
  Análisis de Estabilidad y Correlación de Tendencias

Sube los 4 archivos CSV de validación cruzada:



Saving miflora_log_validacion_cruzada_4.csv to miflora_log_validacion_cruzada_4 (3).csv
Saving miflora_log_validacion_cruzada_3.csv to miflora_log_validacion_cruzada_3 (3).csv
Saving miflora_log_validacion_cruzada_2.csv to miflora_log_validacion_cruzada_2 (3).csv
Saving miflora_log_validacion_cruzada_1.csv to miflora_log_validacion_cruzada_1 (3).csv

 Sesión 1: Mañana - Sin malla
Registros: 80 | Sensores: 4
Período: 2025-12-17 08:48 → 09:15

──────────────────────────────────────────────────
📊 Temperatura (°C)
──────────────────────────────────────────────────

▸ Estabilidad (CV por sensor):
   SensorTomate1: CV = 15.43%  (μ = 14.95 ± 2.31)
   SensorTomate2: CV = 15.32%  (μ = 14.59 ± 2.24)
   SensorTomate3: CV = 10.80%  (μ = 13.83 ± 1.49)
   SensorTomate4: CV = 9.96%  (μ = 13.76 ± 1.37)

   → CV promedio: 12.88%

▸ Correlación de tendencias (Spearman ρ):
   SensorTomate1 ↔ SensorTomate2: ρ = 0.989***
   SensorTomate1 ↔ SensorTomate3: ρ = 0.979***
   SensorTomate1 ↔ SensorTomate4: ρ = 0